You just got hired as the first and only data practitioner at a small business experiencing exponential growth. The company needs more structured processes, guidelines, and standards. Your first mission is to structure the human resources data. The data is currently scattered across teams and files and comes in various formats: Excel files, CSVs, JSON files...

You'll work with the following data in the `datasets` folder:
- __Office addresses__
    - Saved in `office_addresses.csv`. 
    - If the value for office is `NaN`, then the employee is remote.
- __Employee addresses__
    - Saved on the first tab of `employee_information.xlsx`.
- __Employee emergency contacts__ 
    - Saved on the second tab of `employee_information.xlsx`; this tab is called `emergency_contacts`. 
    - However, this sheet was edited at some point, and ***the headers were removed***! The HR manager let you know that they should be: `employee_id`, `last_name`, `first_name`, `emergency_contact`, `emergency_contact_number`, and `relationship`.
- __Employee roles, teams, and salaries__ 
    - This information has been exported from the company's human resources management system into a JSON file titled `employee_roles.json`. Here are the first few lines of that file:
```
{"A2R5H9":
  {
    "title": "CEO",
    "monthly_salary": "$4500",
    "team": "Leadership"
  },
 ...
}
```

In [283]:
import pandas as pd
e_inf = pd.read_excel('datasets/employee_information.xlsx')
e_inf2 = pd.read_excel('datasets/employee_information.xlsx',sheet_name = 1)
e_rol = pd.read_json('datasets/employee_roles.json')
e_rol = e_rol.transpose()
off_add = pd.read_csv('datasets/office_addresses.csv')

In [284]:
e_inf

,employee_id,employee_last_name,employee_first_name,employee_country,employee_city,employee_street,employee_street_number
0,A2R5H9,Hunman,Jax,BE,Leuven,Grote Markt,9
1,H8K0L6,Siff,Tara,GB,London,Baker Street,221
2,G4R7V0,Sagal,Gemma,US,New-York,Perry Street,66
3,M1Z7U9,Coates,Tig,FR,Paris,Rue de l'Université,7


In [285]:
cols_emp = ['employee_first_name', 'employee_last_name',
            'employee_country','employee_city',
           'employee_street','employee_street_number']
e_inf_sub = e_inf[cols_emp]


In [286]:
e_inf_sub.index = e_inf['employee_id']
e_inf_sub.head()

,employee_first_name,employee_last_name,employee_country,employee_city,employee_street,employee_street_number
employee_id,,,,,,
A2R5H9,Jax,Hunman,BE,Leuven,Grote Markt,9
H8K0L6,Tara,Siff,GB,London,Baker Street,221
G4R7V0,Gemma,Sagal,US,New-York,Perry Street,66
M1Z7U9,Tig,Coates,FR,Paris,Rue de l'Université,7


In [287]:
headers = ['employee_id', 'last_name', 'first_name',
'emergency_contact', 'emergency_contact_number',  'relationship']
row = []
for col in e_inf2:
    row.append(col)
e_inf2.loc[-1] = row
e_inf2.rename(columns={row[0] : headers[0],
                       row[1] : headers[1],
                       row[2] : headers[2],
                       row[3] : headers[3],
                       row[4] : headers[4],
                       row[5] : headers[5],}, inplace=True)
e_inf2.index = e_inf.index + 1
e_inf2 = e_inf2.sort_index().reset_index(drop=True)
row

['A2R5H9', 'Hunman', 'Jax', 'Opie Hurst', '+32-456-5556-84', 'Brother']

In [288]:
e_inf2_sub = e_inf2.drop(['employee_id','last_name','first_name'], axis=1)
e_inf2_sub. index = e_inf2['employee_id']
e_inf2_sub

,emergency_contact,emergency_contact_number,relationship
employee_id,,,
H8K0L6,Wendy de Matteo,+44-020-5554-333,Sister
G4R7V0,John Newmark,+1-202-555-194,Husband
M1Z7U9,Venus Noone,+1-202-555-0130,Wife
A2R5H9,Opie Hurst,+32-456-5556-84,Brother


In [289]:
e_rol[['monthly_salary','team','title']]

,monthly_salary,team,title
A2R5H9,$4500,Leadership,CEO
H8K0L6,$4500,Leadership,CFO
G4R7V0,$3000,Sales,Business Developer
M1Z7U9,$2000,People Operations,Office Manager


In [290]:
off_add.head()

,office,office_country,office_city,office_street,office_street_number
0,Leuven Office,BE,Leuven,Martelarenlaan,38
1,ESB Office,US,New York City,Fifth Avenue,350
2,WeWork Office,GB,London,Old Street,207


In [291]:
employees = pd.DataFrame(pd.concat([e_inf_sub,e_inf2_sub,
                                          e_rol[['monthly_salary','team','title']]
                                         ], axis = 1))
employees_final = pd.merge(employees,
                            off_add,
                           left_on = 'employee_country', right_on ='office_country', how='outer')
employees_final = employees_final.fillna('Remote')
employees_final.index = e_inf['employee_id']
employees_final

,employee_first_name,employee_last_name,employee_country,employee_city,employee_street,employee_street_number,emergency_contact,emergency_contact_number,relationship,monthly_salary,team,title,office,office_country,office_city,office_street,office_street_number
employee_id,,,,,,,,,,,,,,,,,
A2R5H9,Jax,Hunman,BE,Leuven,Grote Markt,9,Opie Hurst,+32-456-5556-84,Brother,$4500,Leadership,CEO,Leuven Office,BE,Leuven,Martelarenlaan,38.0
H8K0L6,Tara,Siff,GB,London,Baker Street,221,Wendy de Matteo,+44-020-5554-333,Sister,$4500,Leadership,CFO,WeWork Office,GB,London,Old Street,207.0
G4R7V0,Gemma,Sagal,US,New-York,Perry Street,66,John Newmark,+1-202-555-194,Husband,$3000,Sales,Business Developer,ESB Office,US,New York City,Fifth Avenue,350.0
M1Z7U9,Tig,Coates,FR,Paris,Rue de l'Université,7,Venus Noone,+1-202-555-0130,Wife,$2000,People Operations,Office Manager,Remote,Remote,Remote,Remote,Remote
